# Network compression

Network compression for the computation of EFMs, allowing for greatly reducing the number of model reactions, using some code done by Marco Terzer for EFMTool, and the Stephen Klamt team for the `efmtool_link` python package which is used in CNApy.

In [1]:
import numpy, cobra
import efmtool_link.efmtool_intern as efmtool_intern
import efmtool_link.efmtool_extern as efmtool_extern
import efmtool_link.efmtool4cobra as efmtool4cobra

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [2]:
# all parameters here for simpler utilisation
model_file = 'models_storage/Hep-G2_v2.xml' # model file name
dir_name = './' # parent directory of the directory where mparser_cli is
module_name = './' # directory where mparser_cli is
generated_files_path = '../compressed/' # path where the generated files go
model_name = 'new' # prefix name for all generated files (do not put a path there)

In [316]:
# example of filling the values (put absolute paths, not relative paths)
model_file = '/home/maxime/Bureau/Thesis/asp-efm/data/sbml/mitocore/mitocore.xml' # model file name
dir_name = '/home/maxime/Bureau/Thesis/' # parent directory of the directory where mparser_cli is
module_name = 'asp-efm' # directory where mparser_cli is
generated_files_path = '/home/maxime/Bureau/Thesis/asp-efm/data/sbml/mitocore/' # path where the generated files go
model_name = 'mitocore_rd' # prefix name for all generated files (do not put a path there)

In [3]:
from pathlib import Path
def flux_variability_analysis(model: cobra.Model, loopless=False, fraction_of_optimum=0.0,
                              processes=None, results_cache_dir: Path=None, fva_hash=None, print_func=print):
    # all bounds in the model must be finite because the COBRApy FVA treats unbounded results as errors
    if results_cache_dir is not None:
        fva_hash.update(pickle.dumps((loopless, fraction_of_optimum, model.tolerance))) # integrate solver tolerances?
        if fraction_of_optimum > 0:
            fva_hash.update(pickle.dumps(model.reactions.list_attr("objective_coefficient")))
            fva_hash.update(model.objective_direction.encode())
        file_path = results_cache_dir / (model.id+"_FVA_"+fva_hash.hexdigest())
        fva_result = None
        if Path.exists(file_path):
            try:
                fva_result = pandas.read_pickle(file_path)
                print_func("Loaded FVA result from", str(file_path))
            except:
                print_func("Loading FVA result from", str(file_path), "failed, running FVA.")
        else:
            print_func("No cached result available, running FVA...")
        if fva_result is None:
            fva_result = cobra.flux_analysis.flux_variability_analysis(model, reaction_list=None, loopless=loopless,
                                                             fraction_of_optimum=fraction_of_optimum,
                                                             pfba_factor=None, processes=processes)
            try:
                fva_result.to_pickle(file_path)
                print_func("Saved FVA result to ", str(file_path))
            except:
                print_func("Failed to write FVA result to ", str(file_path))
        return fva_result
    else:
        return cobra.flux_analysis.flux_variability_analysis(model, reaction_list=None, loopless=loopless,
                                                             fraction_of_optimum=fraction_of_optimum,
                                                             pfba_factor=None, processes=processes)

In [4]:
def fva_model(compressed_model):
    fva_tolerance=1e-9
    with model as fva: # can be skipped when a compressed model is available
        # when include_model_bounds=False modify bounds so that only reversibilites are used?
        # fva.solver = 'glpk_exact' # too slow for large models
        fva.tolerance = fva_tolerance
        fva.objective = model.problem.Objective(0.0)
        if fva.problem.__name__ == 'optlang.glpk_interface':
            # should emulate setting an optimality tolerance (which GLPK simplex does not have)
            fva.solver.configuration._smcp.meth = GLP_DUAL
            fva.solver.configuration._smcp.tol_dj = fva_tolerance
        elif fva.problem.__name__ == 'optlang.coinor_cbc_interface':
            fva.solver.problem.opt_tol = fva_tolerance
        fva_res = flux_variability_analysis(fva, fraction_of_optimum=0.0, processes=1, 
            results_cache_dir=None, fva_hash=None)
    for i in range(fva_res.values.shape[0]): # assumes the FVA results are ordered same as the model reactions
        if abs(fva_res.values[i, 0]) > fva_tolerance: # resolve with glpk_exact?
            compressed_model.reactions[i].lower_bound = fva_res.values[i, 0]
        else:
            compressed_model.reactions[i].lower_bound = 0
        if abs(fva_res.values[i, 1]) > fva_tolerance: # resolve with glpk_exact?
            compressed_model.reactions[i].upper_bound = fva_res.values[i, 1]
        else:
            compressed_model.reactions[i].upper_bound = 0
    return compressed_model

In [5]:
model = cobra.io.read_sbml_model(model_file)
original_model = model.copy()

In [8]:
# network compression (currently combination of reaction subsets only)
# IMPORTANT: the model is modified by this function, if you want to keep the full model copy it first
model = fva_model(model)
subT = efmtool4cobra.compress_model_sympy(model) # subT is a matrix for conversion of flux vectors between the full and compressed model
rd = cobra.util.array.create_stoichiometric_matrix(model, array_type='lil')
# model compression makes sure that irreversible reactions always point in the forward direction
rev_rd = [int(r.reversibility) for r in model.reactions]
len(model.reactions)

Flipped EX_m00269x
Flipped EX_m00970x
Flipped EX_m01020x
Flipped EX_m01100x
Flipped EX_m01286x
Flipped EX_m01288x
Flipped EX_m01370x
Flipped EX_m01383x
Flipped EX_m01400x
Flipped EX_m01438x
Flipped EX_m01587x
Flipped EX_m01588x
Flipped EX_m01629x
Flipped EX_m01714x
Flipped EX_m01716x
Flipped EX_m01743x
Flipped EX_m01745x
Flipped EX_m01796x
Flipped EX_m01821x
Flipped EX_m01913x
Flipped EX_m01962x
Flipped EX_m02034x
Flipped EX_m02104x
Flipped EX_m02105x
Flipped EX_m02125x
Flipped EX_m02136x
Flipped EX_m02184x
Flipped EX_m02193x
Flipped EX_m02278x
Flipped EX_m02332x
Flipped EX_m02353x
Flipped EX_m02354x
Flipped EX_m02360x
Flipped EX_m02362x
Flipped EX_m02387x
Flipped EX_m02389x
Flipped EX_m02426x
Flipped EX_m02445x
Flipped EX_m02453x
Flipped EX_m02470x
Flipped EX_m02471x
Flipped EX_m02560x
Flipped EX_m02630x
Flipped EX_m02631x
Flipped EX_m02672x
Flipped EX_m02724x
Flipped EX_m02789x
Flipped EX_m02926x
Flipped EX_m02943x
Flipped EX_m02945x
Flipped EX_m02982x
Flipped EX_m02993x
Flipped EX_m

2251

In [9]:
print(sum(rev_rd), 'reversible reactions and', len(rev_rd) - sum(rev_rd), 'irreversible')

562 reversible reactions and 1689 irreversible


In [10]:
import pandas

In [11]:
# Fonction qui, pour chaque élément itérable l, lui applique la fonction qui en éxtrait l'id de ses éléments.
names = lambda l: list(map(lambda x: x.id, l))

In [12]:
df = pandas.DataFrame.sparse.from_spmatrix(rd, index=names(model.metabolites), columns=names(model.reactions))
df

,EX_m00097x,EX_m00157x,EX_m00266x,EX_m00267x,EX_m00268x,EX_m00269x,EX_m00648x,EX_m00970x,EX_m01020x,EX_m01100x,...,HMR_9025,HMR_9027,HMR_9028,HMR_9029,HMR_9030,HMR_9031,HMR_9051,HMR_9055,biomass_components,artificial_biomass
m00003c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
m00003s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
m00004c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
m00006c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
m00007c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
m1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
m2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
m3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
m4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
import io
with io.StringIO() as output:
    for column in df.columns:
        dfc = df[column]
        ftime=True
        print(column, ': ', end='', file=output, flush=True)
        for k, v in dfc[dfc < 0].items():
            if ftime:
                ftime=False
            else:
                print('+', end=' ', file=output, flush=True)
            print(-v, k, end=' ', file=output, flush=True)
        print('=' if model.reactions.get_by_id(column).reversibility else '=>', end=' ', file=output, flush=True)
        ftime=True
        for k, v in dfc[dfc > 0].items():
            if ftime:
                ftime=False
            else:
                print('+', end=' ', file=output, flush=True)
            print(v, k, end=' ', file=output, flush=True)
        print(file=output, flush=True)
    lines_r = output.getvalue()
print(lines_r)

EX_m00097x : 1.0 m00097s = 
EX_m00157x : 1.0 m00157s + 1.0 m01253m + 1.0 m02039m + 1.0 m02553m = 1.0 m00157m + 1.0 m02552m 
EX_m00266x : 1.0 m00267s + 4.0 m02040s => 4.0 m01974s 
EX_m00267x : 1.0 m00267s = 
EX_m00268x : 1.0 m00268s = 
EX_m00269x : 1.0 m02040s => 1.0 m00268s + 1.0 m01974s 
EX_m00648x : 1.0 m00648s + 1.0 m00671c + 1.0 m02039c + 1.0 m02553c = 1.0 m00648c + 1.0 m02552c 
EX_m00970x : => 1.0 m00970s 
EX_m01020x : 1.0 m02040s => 1.0 m02751s 
EX_m01100x : 1.0 m02147c => 1.0 m01100c + 1.0 m02147s 
EX_m01107x : 1.0 m01107c + 0.6666666666666666 m01371c + 0.6666666666666666 m02040c => 0.6666666666666666 m01285c + 0.6666666666666666 m02751c 
EX_m01111x : 1.0 m02366s => 1.0 m02418s 
EX_m01115x : 1.0 m01115c + 1.0 m02147s = 1.0 m02147c 
EX_m01159x : 1.0 m01159s => 
EX_m01252x : 1.0 m01252s = 
EX_m01253x : 1.0 m01253s = 
EX_m01280x : 1.0 m01280s = 
EX_m01285x : 1.0 m01285s = 
EX_m01286x : 1.0 m02040c => 1.0 m01334c + 1.0 m01967c 
EX_m01288x : 1.0 m02040c => 1.0 m01334c + 1.0 m02845c 


In [14]:
rxn_in_sub = [numpy.where(subT[:, i])[0] for i in range(subT.shape[1])]

In [15]:
for i, reactions in enumerate(model.reactions):
    print(f'rsub_{i+1}', list(map(lambda x: original_model.reactions[x].id, rxn_in_sub[i])))

rsub_1 ['EX_m00097x', 'HMR_9338']
rsub_2 ['EX_m00157x', 'HMR_4461', 'HMR_9134']
rsub_3 ['EX_m00266x', 'HMR_7920', 'HMR_9205']
rsub_4 ['EX_m00267x', 'HMR_9206']
rsub_5 ['EX_m00268x', 'HMR_9207']
rsub_6 ['EX_m00269x', 'HMR_7922', 'HMR_9208']
rsub_7 ['EX_m00648x', 'HMR_5351', 'HMR_9216']
rsub_8 ['EX_m00970x', 'HMR_9091']
rsub_9 ['EX_m01020x', 'EX_m02754x', 'HMR_9569', 'HMR_9101', 'HMR_9682']
rsub_10 ['EX_m01100x', 'HMR_7682', 'HMR_9100']
rsub_11 ['EX_m01107x', 'HMR_7634', 'HMR_9094']
rsub_12 ['EX_m01111x', 'HMR_1084', 'HMR_9683']
rsub_13 ['EX_m01115x', 'HMR_7683', 'HMR_9234']
rsub_14 ['EX_m01159x', 'HMR_9330']
rsub_15 ['EX_m01252x', 'HMR_9086']
rsub_16 ['EX_m01253x', 'HMR_9132']
rsub_17 ['EX_m01280x', 'HMR_9254']
rsub_18 ['EX_m01285x', 'HMR_9255']
rsub_19 ['EX_m01286x', 'HMR_8443', 'HMR_9183', 'HMR_9456']
rsub_20 ['EX_m01288x', 'HMR_7144', 'HMR_7905', 'HMR_9141']
rsub_21 ['EX_m01306x', 'HMR_9259']
rsub_22 ['EX_m01307x', 'HMR_9061']
rsub_23 ['EX_m01308x', 'HMR_5198', 'HMR_9730']
rsub_24 ['

In [16]:
lines = ''
nb_ext = 1
for i, column in enumerate(df.columns):
    dfc = df[column]
    ftime=True
    lines += f'rsub_{i+1}' + ' : '
    for k, v in dfc[dfc < 0].items():
        if ftime:
            ftime=False
        else:
            lines += ' + '
        lines += str(-v) + ' ' + k
    if ftime:
        lines += f'ext_{nb_ext}'
        nb_ext += 1
    lines += ' = ' if model.reactions.get_by_id(column).reversibility else ' => '
    ftime=True
    for k, v in dfc[dfc > 0].items():
        if ftime:
            ftime=False
        else:
            lines += ' + '
        lines += str(v) + ' ' + k
    if ftime:
        lines += f'ext_{nb_ext}'
        nb_ext += 1
    lines += '\n'
lines = '-METEXT\n' + ' '.join(['ext_' + str(i+1) for i in range(nb_ext-1)]) + '\n' + '\n-CAT\n' + lines
print(lines)

-METEXT
ext_1 ext_2 ext_3 ext_4 ext_5 ext_6 ext_7 ext_8 ext_9 ext_10 ext_11 ext_12 ext_13 ext_14 ext_15 ext_16 ext_17 ext_18 ext_19 ext_20 ext_21 ext_22 ext_23 ext_24 ext_25 ext_26 ext_27 ext_28 ext_29 ext_30 ext_31 ext_32 ext_33 ext_34 ext_35 ext_36 ext_37 ext_38 ext_39 ext_40 ext_41 ext_42 ext_43 ext_44 ext_45 ext_46 ext_47 ext_48 ext_49 ext_50 ext_51 ext_52 ext_53 ext_54 ext_55 ext_56 ext_57 ext_58 ext_59 ext_60 ext_61 ext_62 ext_63 ext_64 ext_65 ext_66 ext_67 ext_68 ext_69 ext_70 ext_71 ext_72 ext_73 ext_74 ext_75 ext_76 ext_77 ext_78 ext_79 ext_80 ext_81 ext_82 ext_83 ext_84 ext_85 ext_86 ext_87 ext_88 ext_89 ext_90 ext_91 ext_92 ext_93 ext_94 ext_95 ext_96 ext_97 ext_98 ext_99 ext_100 ext_101 ext_102 ext_103 ext_104 ext_105 ext_106 ext_107 ext_108 ext_109 ext_110 ext_111 ext_112

-CAT
rsub_1 : 1.0 m00097s = ext_1
rsub_2 : 1.0 m00157s + 1.0 m01253m + 1.0 m02039m + 1.0 m02553m = 1.0 m00157m + 1.0 m02552m
rsub_3 : 1.0 m00267s + 4.0 m02040s => 4.0 m01974s
rsub_4 : 1.0 m00267s = ext_2

In [17]:
dicto_rsub = {}
biomass = f'rsub_1'
for i, reactions in enumerate(model.reactions):
    lm = list(map(lambda x: original_model.reactions[x].id, rxn_in_sub[i]))
    if any(('biomass' in k) or ('BIOMASS' in k) or ('Biomass' in k) for k in lm):
        biomass = f'rsub_{i+1}'
    for r in lm:
        dicto_rsub[r] = f'rsub_{i+1}'
print(biomass)

rsub_2251


In [18]:
biomass = biomass # change biomass here if not good
print(biomass)

rsub_2251


In [19]:
for reaction in model.reactions:
    print(reaction.id, reaction.lower_bound, reaction.upper_bound, reaction.subset_stoich)

EX_m00097x -1000.0 1000.0 [ 1. -1.]
EX_m00157x -1000.0 1000.0 [ 1. -1. -1.]
EX_m00266x 0 750.0 [1. 1. 1.]
EX_m00267x -750.0 1000.0 [ 1. -1.]
EX_m00268x -1000.0 1000.0 [ 1. -1.]
EX_m00269x 0 1000.0 [1. 1. 1.]
EX_m00648x -1000.0 1000.0 [ 1. -1. -1.]
EX_m00970x 0 1000.0 [1. 1.]
EX_m01020x 0 1000.0 [1. 1. 1. 1. 1.]
EX_m01100x 0 1000.0 [1. 1. 1.]
EX_m01107x 0 1000.0 [1.         0.33333333 1.        ]
EX_m01111x 0 1000.0 [1. 1. 1.]
EX_m01115x -1000.0 1000.0 [ 1.  1. -1.]
EX_m01159x 0 1000.0 [1. 1.]
EX_m01252x -1000.0 1000.0 [ 1. -1.]
EX_m01253x -1000.0 1000.0 [ 1. -1.]
EX_m01280x -1000.0 1000.0 [ 1. -1.]
EX_m01285x -1000.0 1000.0 [ 1. -1.]
EX_m01286x 0 1000.0 [1. 1. 1. 1.]
EX_m01288x 0 1000.0 [1. 1. 1. 1.]
EX_m01306x -1000.0 1000.0 [ 1. -1.]
EX_m01307x -1000.0 1000.0 [ 1. -1.]
EX_m01308x -1.4429181576820955 5.712691038626327 [-1. -1.  1.]
EX_m01334x -1000.0 1000.0 [ 1. -1.]
EX_m01365x -42.486919859929685 1000.0 [ 1. -1.]
EX_m01369x -1000.0 1000.0 [ 1. -1.]
EX_m01370x 0 1000.0 [1. 1.]
EX_m013

# File creation

In [20]:
from pathlib import Path
from os import chdir
chdir(dir_name)

In [21]:
import sys
sys.path.append(module_name)
import mparser

In [22]:
path = generated_files_path

In [23]:
txtfile = path + f'{model_name}_reduced.txt'

In [24]:
with open(txtfile, 'w') as f:
    f.writelines(lines)

In [25]:
ri = lambda i: original_model.reactions[i].id

In [26]:
subname = lambda i: 'rsub_{}'.format(i+1)
subnamerev = lambda i: 'rsub_{}_rev'.format(i+1)
aq = lambda x: '"{}"'.format(x)
aqrev = lambda x: '"{}_rev"'.format(x)
rev = lambda x: '{}_rev'.format(x)  
lines = 'subset(' + ';'.join([aq(subname(i)) for i in range(len(df.columns))]) + ').\n'
lines += 'subset(' + ';'.join([aq(subnamerev(i)) for i in range(len(df.columns))]) + ').\n'
dicto = {}

for r, ls in enumerate(rxn_in_sub):
    dicto[subname(r)] = {'reacs': [ri(rk) if subT[rk, r] >= 0 else rev(ri(rk)) for rk in ls],
         'coeffs': [abs(subT[rk, r]) for rk in ls],
         'full': [(ri(rk), subT[rk, r]) for rk in ls]}
    if model.reactions.get_by_any(r)[0].reversibility:
        dicto[subnamerev(r)] = {'reacs': [ri(rk) if subT[rk, r] < 0 else rev(ri(rk)) for rk in ls],
             'coeffs': [abs(subT[rk, r]) for rk in ls],
             'full': [(ri(rk), -subT[rk, r]) for rk in ls]}
    for rk in ls:
        fk = subT[rk, r]
        lines += 'coefficient(' + aq(subname(r)) + ',' + ((aq(ri(rk))) if fk >= 0 else aqrev(ri(rk))) + ',' + aq(abs(fk)) + ').\n'
        lines += 'coefficient(' + aq(subnamerev(r)) + ',' + ((aqrev(ri(rk))) if fk >= 0 else aq(ri(rk))) + ',' + aq(abs(fk)) + ').\n'

In [27]:
lp4file = path + f'{model_name}_reactionSubsets.lp4'
subfile = path + f'{model_name}_reactionSubsets.txt'

In [28]:
with open(lp4file, 'w') as f:
    f.writelines(lines)

In [29]:
with open(subfile, 'w') as f:
    f.write(str(dicto))

In [30]:
mcssubname = lambda i: '"mcs_rsub_{}"'.format(i+1)
mcssubnamerev = lambda i: '"mcs_rsub_{}_rev"'.format(i+1)
aq = lambda x: '"{}"'.format(x)
aqrev = lambda x: '"{}_rev"'.format(x)    
lines = 'subset(' + ';'.join([mcssubname(i) for i in range(len(df.columns))]) + ').\n'
lines += 'subset(' + ';'.join([mcssubnamerev(i) for i in range(len(df.columns))]) + ').\n'

for r, ls in enumerate(rxn_in_sub):
    for rk in ls:
        fk = subT[rk, r]
        lines += 'coefficient(' + mcssubname(r) + ',' + ((aq(ri(rk))) if fk >= 0 else aqrev(ri(rk))) + ',' + aq(abs(fk)) + ').\n'
        lines += 'coefficient(' + mcssubnamerev(r) + ',' + ((aqrev(ri(rk))) if fk >= 0 else aq(ri(rk))) + ',' + aq(abs(fk)) + ').\n'

In [31]:
lp4file = path + f'{model_name}_mcsReactionSubsets.lp4'

In [32]:
with open(lp4file, 'w') as f:
    f.writelines(lines)

In [33]:
len(original_model.reactions)

5906

In [34]:
txtfile=f'{path}{model_name}_reduced.txt'
#pklfile=f'{path}{model_name}_reduced.pkl'
aspfile=f'{path}{model_name}_reduced.lp4'
sbmlfile=f'{path}{model_name}_reduced.xml'
#mcs_pklfile=f'{path}{model_name}_reduced_mcs.pkl'
mcs_aspfile=f'{path}{model_name}_reduced_mcs.lp4'
#mcs_sbmlfile=f'{path}{model_name}_reduced_mcs.xml'

### Compressed network files for EFMs computation

In [35]:
#mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.PKL, pklfile, False, [])
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.ASP, aspfile, False, [])
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.SBML, sbmlfile, False, [])

/home/jmuller/Documents/Stage_metabo/mparser/meta_network.py:45: UserWarning: Found reactions with missing internal metabolite production or consumption
  warnings.warn('Found reactions with missing internal metabolite production or consumption')


893 internal metabolites
112 external metabolites
2251 total reactions
103 exchange reactions
562 reversible reactions
893 internal metabolites
112 external metabolites
2251 total reactions
103 exchange reactions
562 reversible reactions


### Compressed network files for MCSs computation with 'biomass' as target

In [36]:
#mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.PKL, mcs_pklfile, to_dual_mcs=True, target_reactions=[biomass])
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.ASP, mcs_aspfile, to_dual_mcs=True, target_reactions=[biomass])
#mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.SBML, mcs_sbmlfile, to_dual_mcs=True, target_reactions=[biomass])

893 internal metabolites
112 external metabolites
2251 total reactions
103 exchange reactions
562 reversible reactions


/home/jmuller/Documents/Stage_metabo/mparser/meta_network.py:203: UserWarning: No external metabolite
  self.structures_check()
/home/jmuller/Documents/Stage_metabo/mparser/meta_network.py:43: UserWarning: Found reactions with missing overall metabolite production or consumption
  warnings.warn('Found reactions with missing overall metabolite production or consumption')


-- Dual network --
2251 internal metabolites
0 external metabolites
3145 total reactions
103 exchange reactions
1455 reversible reactions
